In [7]:
from pandas_gbq import to_gbq
from google.oauth2 import service_account
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import logging
from dotenv import load_dotenv
import os

In [8]:
load_dotenv()

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
credencial = service_account.Credentials.from_service_account_file(
    os.getenv('GOOGLE_APPLICATION_CREDENTIALS'),
    scopes=["https://www.googleapis.com/auth/bigquery"])

In [9]:
driver = webdriver.Chrome()

driver.get("https://steamdb.info/sales/")


In [10]:
#Função scroll down para atualizar os dados que vão ser extraídos
def scroll_down_page(speed=10):
        current_scroll_position, new_height = 0, 1
        while current_scroll_position <= new_height:
            current_scroll_position += speed
            driver.execute_script(
                "window.scrollTo(0, {});".format(current_scroll_position))
            new_height = driver.execute_script(
                "return document.body.scrollHeight")

In [11]:
#Cria uma lista vazia aonde serão armazenados os dados e cria a função de extração
games = []

def extract_data_from_page(soup):
    table = soup.find(id="DataTables_Table_0")
    rows = table.find_all("tr", class_="app")

    for row in rows:
        data_cells = row.find_all("td")

        game_data = {
            "Name": data_cells[2].find("a").text.strip(),
            "Discount_percentage": data_cells[3].text.strip(),
            "Price": data_cells[4].text.strip(),
            "Rating": data_cells[5].text.strip(),
            "Release": data_cells[6].text.strip(),
            "Ends": data_cells[7].text.strip(),
            "Started": data_cells[8].text.strip()
        }

        logging.info(game_data)
        games.append(game_data)

In [12]:
while True:
        scroll_down_page()  # Rolando lentamente até que todas as linhas estejam visíveis
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        extract_data_from_page(soup)

        try:
            next_button = driver.find_element(
                By.CSS_SELECTOR, 'button.dt-paging-button.next')
            if next_button.get_attribute('aria-disabled') == 'true':
                break
            actions = ActionChains(driver)
            actions.move_to_element(next_button).click().perform()
            #next_button.click()
            WebDriverWait(driver, 10).until(EC.staleness_of(next_button))
        except Exception as e:
            logging.error(f"Erro ao navegar: {e}")
            break


2024-11-10 17:19:37,162 - INFO - {'Name': 'Pistol Whip', 'Discount_percentage': '-50%', 'Price': 'R$ 44,49', 'Rating': '90.35%', 'Release': 'Nov 2019', 'Ends': 'in 22 hours', 'Started': '2 days ago'}
2024-11-10 17:19:37,164 - INFO - {'Name': 'Age of Wonders 4', 'Discount_percentage': '-35%', 'Price': 'R$ 129,99', 'Rating': '79.35%', 'Release': 'May 2023', 'Ends': 'in 4 days', 'Started': '3 days ago'}
2024-11-10 17:19:37,165 - INFO - {'Name': 'ICARUS', 'Discount_percentage': '-50%', 'Price': 'R$ 50,99', 'Rating': '70.14%', 'Release': 'Dec 2021', 'Ends': 'in 4 days', 'Started': '3 days ago'}
2024-11-10 17:19:37,165 - INFO - {'Name': 'Age of Water', 'Discount_percentage': '-50%', 'Price': 'R$ 74,50', 'Rating': '62.28%', 'Release': 'Apr 2024', 'Ends': 'in 4 days', 'Started': '3 days ago'}
2024-11-10 17:19:37,166 - INFO - {'Name': "Baldur's Gate 3", 'Discount_percentage': '-20%', 'Price': 'R$ 159,99', 'Rating': '95.98%', 'Release': 'Aug 2023', 'Ends': 'in 22 hours', 'Started': '6 days ago'}

In [13]:
# Cria o DataFrame
df = pd.DataFrame(games)

logging.info("DataFrame criado com sucesso")

2024-11-10 17:22:39,473 - INFO - DataFrame criado com sucesso


In [14]:
# Exporta os dados para o BigQuery
try:
    df.to_gbq(destination_table=os.getenv('BIGQUERY_TABLE'),
              project_id=os.getenv('BIGQUERY_PROJECT_ID'),
              if_exists='replace',
              credentials=credencial)
    logging.info("Dados carregados com sucesso no BigQuery")
except Exception as e:
    logging.error(f"Erro ao carregar dados no BigQuery: {e}")

C:\Users\Pedro\AppData\Local\Temp\ipykernel_34192\2938485854.py:3: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=os.getenv('BIGQUERY_TABLE'),
1914 out of 1914 rows loaded.O - 
2024-11-10 17:22:52,537 - INFO - Dados carregados com sucesso no BigQuery


In [15]:
display(df)

,Name,Discount_percentage,Price,Rating,Release,Ends,Started
0,Pistol Whip,-50%,"R$ 44,49",90.35%,Nov 2019,in 22 hours,2 days ago
1,Age of Wonders 4,-35%,"R$ 129,99",79.35%,May 2023,in 4 days,3 days ago
2,ICARUS,-50%,"R$ 50,99",70.14%,Dec 2021,in 4 days,3 days ago
3,Age of Water,-50%,"R$ 74,50",62.28%,Apr 2024,in 4 days,3 days ago
4,Baldur's Gate 3,-20%,"R$ 159,99",95.98%,Aug 2023,in 22 hours,6 days ago
...,...,...,...,...,...,...,...
1909,Yacht Mechanic Simulator,-34%,"R$ 44,89",26.18%,Jul 2022,—,4 days ago
1910,Time Ramesside (A New Reckoning),-90%,"R$ 1,74",25.64%,May 2015,in 9 days,5 days ago
1911,Bomb Defense,-50%,"R$ 5,44",25.60%,Jul 2017,in 22 hours,6 days ago
1912,Lords of the Black Sun,-82%,"R$ 13,31",24.88%,Sep 2014,in 22 hours,6 days ago
